# Code-to-Code Comparison: Dinwoodie

National Renewable Energy Laboratory\
Rob Hammond\
10 January 2023

**Note**: The results in this notebook will differ from those in prior versions quite significantly in some places due to the numerous logic changes that have both updated and improved the features provided by WOMBAT, and fixed previously unidentified bugs.

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat import Simulation
from wombat.core.library import DINWOODIE, load_yaml

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base",
    "more_ctvs",
    "fewer_ctvs",
    "more_techs",
    "fewer_techs",
    "failure_50",
    "failure_200",
    "no_hlvs",
    "no_weather",
    "historic_weather",
    "manual_resets_only",
    "minor_repairs_only",
    "medium_repairs_only",
    "major_repairs_only",
    "major_replacements_only",
    "annual_service_only",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
    
}
metrics_dict = {}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(34)} | {(end - start) / 60:.2f} m")
    metrics_dict[config] = sim.metrics

                    dinwoodie_base | 0.70 m
               dinwoodie_more_ctvs | 0.65 m
              dinwoodie_fewer_ctvs | 0.68 m
              dinwoodie_more_techs | 0.63 m
             dinwoodie_fewer_techs | 0.67 m
              dinwoodie_failure_50 | 0.63 m
             dinwoodie_failure_200 | 0.73 m
                 dinwoodie_no_hlvs | 0.76 m
              dinwoodie_no_weather | 0.90 m
        dinwoodie_historic_weather | 0.65 m
      dinwoodie_manual_resets_only | 0.59 m
      dinwoodie_minor_repairs_only | 0.59 m
     dinwoodie_medium_repairs_only | 0.51 m
      dinwoodie_major_repairs_only | 0.50 m
 dinwoodie_major_replacements_only | 0.52 m
     dinwoodie_annual_service_only | 0.59 m


In [5]:
for config, metrics in metrics_dict.items():
    # Gather the results of interest
    years = metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability_time = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
    availability_production = metrics.production_based_availability(frequency="project", by="windfarm").values[0][0]
    cf_net = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
    cf_gross = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
    power_production = metrics.power_production(frequency="project", by="windfarm").values[0][0]
    completion_rate = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
    parts = metrics.events[["materials_cost"]].sum().sum()
    techs = metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    total = metrics.events[["total_cost"]].sum().sum()
    
    equipment = metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]].sum().sum()
    fsv = equipment[[el for el in equipment.columns if "Field Support Vessel" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    
    utilization = metrics.service_equipment_utilization(frequency="project")
    hlv_ur = utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]].mean().mean()
    fsv_ur = utilization[[el for el in utilization.columns if "Field Support Vessel" in el]].mean().mean()
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)

/Users/rhammond/Documents/GitHub/WOMBAT/wombat/core/post_processor.py:862: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  operating_df = operating_df.append(missing).sort_index()


In [6]:
# Save the results
with open(DINWOODIE / "results" / "results_dict_v0.6.0.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(DINWOODIE / "results" / "results_data_v0.6.0.csv", index_label="result")

In [7]:
results_df

,base,more_ctvs,fewer_ctvs,more_techs,fewer_techs,failure_50,failure_200,no_hlvs,no_weather,historic_weather,manual_resets_only,minor_repairs_only,medium_repairs_only,major_repairs_only,major_replacements_only,annual_service_only
availability - time based,0.96,0.95,0.96,0.96,0.95,0.97,0.93,0.97,0.97,0.95,0.99,1.00,1.00,1.00,0.97,0.99
availability - production based,0.96,0.95,0.96,0.96,0.95,0.97,0.93,0.97,0.00,0.95,0.99,1.00,1.00,1.00,0.97,0.99
capacity factor - net,0.46,0.45,0.46,0.46,0.45,0.46,0.44,0.46,0.00,0.46,0.47,0.47,0.48,0.48,0.46,0.47
capacity factor - gross,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.00,0.48,0.48,0.48,0.48,0.48,0.48,0.48
power production,"9,613.08","9,539.82","9,618.90","9,620.84","9,550.20","9,689.75","9,358.36","9,760.20",0.00,"7,721.08","9,977.30","9,984.31","10,014.89","10,027.12","9,756.76","9,886.40"
task completion rate,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.96,1.00
annual direct O&M cost,11.31,15.62,12.19,14.12,11.44,8.99,15.68,5.39,10.72,12.47,3.48,3.73,3.91,3.97,11.90,4.81
annual vessel cost,7.18,8.85,7.74,8.82,7.88,5.21,10.52,1.82,6.68,7.88,1.87,1.89,1.91,2.14,8.73,1.88
ctv cost,1.68,3.11,1.85,1.86,1.85,1.86,1.85,1.82,1.74,1.84,1.87,1.89,1.91,1.92,1.92,1.88
fsv cost,0.19,0.19,0.16,0.17,0.16,0.08,0.21,0.00,0.15,0.12,0.00,0.00,0.00,0.22,0.00,0.00
